In [6]:
import numpy as np

In [1]:
def f(x1,x2):
    return 3**(1/(x1,x2))+x1**2+x2**2

In [10]:
def Gradiente(x1,x2):
    return [[2*x1-(np.log(3)*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**2],
            [2*x2-(np.log(3)*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**2]]

In [11]:
def Heissiana(x1,x2):
    return [[2 - (np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2))))/(x1+x2)**4, -np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**4],

            [-np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**4 ,2 - (np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2))*(x1+x2)))/(x1+x2)**4]]